# Importing required libraries
- pandas for segregating and cleaning the data
- numpy for array handling

### Issues faced
- datetime in pandas and tensorflow (*which has to be added*)

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import xlrd
import csv

In [2]:
# file_location = 'BUSYv1.csv'
file_location = 'BUSY7246fb6.csv'
df = pd.read_csv(file_location, sep='|', names=['POS_Application_Name','STOREID','MACID','BILLNO','BARCODE','GUID','CREATED_STAMP','CAPTURED_WINDOW','UPDATE_STAMP'])
barcode_size_corrected_df = df[(df['BARCODE'].str.len() == 12) | (df['BARCODE'].str.len() == 13) | (df['BARCODE'].str.len() == 8)]
no_capture_window = barcode_size_corrected_df[['POS_Application_Name','STOREID','BARCODE','GUID','CREATED_STAMP']]
barcode_pattern = '^[0-9]*$'
barcode_rectified = no_capture_window[no_capture_window['BARCODE'].str.contains(barcode_pattern)]
len(barcode_rectified.index)

56513

In [3]:
items_sold = barcode_rectified['BARCODE'].value_counts()
print(items_sold[:5])

8901030564185    668
8901030534898    425
8901058842722    363
8901030341663    300
8901030627378    295
Name: BARCODE, dtype: int64


#### Datatype issue

Here BARCODE and CREATED_STAMP have the datatype of **object** which is analogous to **string** in native python

In [4]:
print('barcode_rectified.BARCODE.dtype:\t' + str(barcode_rectified.BARCODE.dtype))
print('barcode_rectified.CREATED_STAMP.dtype:\t' + str(barcode_rectified.CREATED_STAMP.dtype))

barcode_rectified.BARCODE.dtype:	object
barcode_rectified.CREATED_STAMP.dtype:	object


#### Barcode Rectified
Using
```python
pd.to_datetime(<df>, format='<shell format>')
```
we have changed the CREATED_STAMP attribute from object to datetime Dtype

In [5]:
barcode_rectified['CREATED_STAMP'] = pd.to_datetime(barcode_rectified['CREATED_STAMP'], format='%Y-%m-%d %H:%M:%S')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
print('barcode_rectified.CREATED_STAMP.dtype:\t' + str(barcode_rectified.CREATED_STAMP.dtype))

barcode_rectified.CREATED_STAMP.dtype:	datetime64[ns]


In [7]:
# making a COPY for joins and other operations (just to change tha name)
busy_df = barcode_rectified.copy(deep=True)

# Loading the product master DataFrame
pm_df = pd.read_excel('ProductMaster404b8b3.xlsx', header = 0)

pm_df

,COMPANY_CODE,CATEGORY_DESC,SUBCATEGORY_DESC,BRAND_DESC,BASEPACK,BASEPACK_DESC,BARCODE
0,HUL,PFW,Washing Powder,Rin,16040,RIN ADVANCED POWDER NM 4 Kg,8901030648229
1,HUL,PFW,Washing Powder,Rin,16207,RIN ADVANCED POWDER MOGRA NM 4 Kg,8901030648250
2,HUL,CPD,Instant Noodles,Knorr,80759,KNR DRY NOODLES - CHINESE HOT N SPICY,8901030649868
3,HUL,CPD,Instant Noodles,Knorr,80758,KNR DRY NOODLES - CHINESE SCHEZWAN,8901030657627
4,HUL,MPW,Personal Wash Bars,Lifebuoy,15194,LIFEBUOY TOTA10 125g MP,8901030653742
5,HUL,PPW,Personal Wash Bars,PEARS,17336,PEARS PURE AND GENTLE SOAP 3*75G AMBER,8901030652080
6,HUL,CPD,Instant Noodles,Knorr,80758,KNR DRY NOODLES - CHINESE SCHEZWAN,8901030649837
7,HUL,Skin,Fairness,Ponds,12682,Ponds WB Daily Lightening Cream 35 gms,8901030638084
8,HUL,CPD,Ketchup-Sauces,Kissan,80456,Kissan Ketchup -Sweet & Spicy 500Gms,8901030650574
9,HUL,Oral,Toothpaste,Pepsodent,10075,Pepsodent G TP Tube 150g,8901030651144


In [8]:
# Making the BARCODE as object (String)
busy_df.BARCODE = busy_df.BARCODE.astype('int')
# busy_df

# Joining the two DataFrames based on the barcodes and extracting only category_desc, subcategory_desc, brand_desc and basepack_desc
joined_df = busy_df.join(pm_df.set_index('BARCODE'), on='BARCODE', how='inner', sort=False)

# print(joined_df.BARCODE.dtype)
# print(busy_df.BARCODE.dtype)
# print(pm_df.BARCODE.dtype)
joined_df

,POS_Application_Name,STOREID,BARCODE,GUID,CREATED_STAMP,COMPANY_CODE,CATEGORY_DESC,SUBCATEGORY_DESC,BRAND_DESC,BASEPACK,BASEPACK_DESC
1,BUSY,DEL0000001446,8901399058561,a71a95e9-a4a5-4677-9d56-19102598a8ba,2017-01-01 11:55:38.916,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
13695,BUSY,DEL0000001446,8901399058561,94f3cde4-8fe1-4a18-abf3-b3d2a0c2f4c2,2017-01-02 18:05:45.437,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
24018,BUSY,DEL0000001446,8901399058561,d08dfb5f-0801-4fa1-a648-c0dd95d037a6,2017-01-25 17:03:22.006,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
43420,BUSY,DEL0000001446,8901399058561,ef95e2dc-0b28-4858-ab57-a32527b402cc,2017-01-04 21:16:46.908,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
71814,BUSY,DEL0000001446,8901399058561,92e0bc79-599a-4bb5-8065-1d1094d81086,2017-01-08 19:28:35.832,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
115762,BUSY,DEL0000001446,8901399058561,77adc976-bcee-414a-bef2-aa2eed8ba423,2017-01-13 16:56:00.282,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
122086,BUSY,DEL0000001446,8901399058561,41c9b7ce-26f2-4ba5-9fbf-01b34d300cad,2017-01-14 14:36:54.543,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
136645,BUSY,DEL0000001446,8901399058561,9961e86e-1cb9-4dd7-ab72-5d38a3fce68f,2017-01-16 15:29:32.431,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
358903,BUSY,DEL0000001446,8901399058561,7a391c7c-2455-4b20-906e-984f461398da,2017-03-12 19:21:36.353,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML
359028,BUSY,DEL0000001446,8901399058561,c4084a76-657d-4bd3-968a-4f7d2535ad6f,2017-03-12 21:14:16.344,NaN,SOAP,HAND WASH,SANTOOR,SOAP,SANTOOR HAND WASH - GLYCERIN & APRICOT 180ML


In [9]:
# Extracting only required columns
joined_df = joined_df[['BARCODE', 'CREATED_STAMP', 'CATEGORY_DESC', 'SUBCATEGORY_DESC', 'BRAND_DESC', 'BASEPACK']]

joined_df

,BARCODE,CREATED_STAMP,CATEGORY_DESC,SUBCATEGORY_DESC,BRAND_DESC,BASEPACK
1,8901399058561,2017-01-01 11:55:38.916,SOAP,HAND WASH,SANTOOR,SOAP
13695,8901399058561,2017-01-02 18:05:45.437,SOAP,HAND WASH,SANTOOR,SOAP
24018,8901399058561,2017-01-25 17:03:22.006,SOAP,HAND WASH,SANTOOR,SOAP
43420,8901399058561,2017-01-04 21:16:46.908,SOAP,HAND WASH,SANTOOR,SOAP
71814,8901399058561,2017-01-08 19:28:35.832,SOAP,HAND WASH,SANTOOR,SOAP
115762,8901399058561,2017-01-13 16:56:00.282,SOAP,HAND WASH,SANTOOR,SOAP
122086,8901399058561,2017-01-14 14:36:54.543,SOAP,HAND WASH,SANTOOR,SOAP
136645,8901399058561,2017-01-16 15:29:32.431,SOAP,HAND WASH,SANTOOR,SOAP
358903,8901399058561,2017-03-12 19:21:36.353,SOAP,HAND WASH,SANTOOR,SOAP
359028,8901399058561,2017-03-12 21:14:16.344,SOAP,HAND WASH,SANTOOR,SOAP


#### Making data for week wise
- This is done by adding a column for the week number

In [10]:
joined_df['CREATED_STAMP'] = joined_df['CREATED_STAMP'].apply(pd.datetools.normalize_date)

joined_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.core.datetools.normalize_date is deprecated. Please use pandas._libs.tslib.normalize_date instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,BARCODE,CREATED_STAMP,CATEGORY_DESC,SUBCATEGORY_DESC,BRAND_DESC,BASEPACK
1,8901399058561,2017-01-01,SOAP,HAND WASH,SANTOOR,SOAP
13695,8901399058561,2017-01-02,SOAP,HAND WASH,SANTOOR,SOAP
24018,8901399058561,2017-01-25,SOAP,HAND WASH,SANTOOR,SOAP
43420,8901399058561,2017-01-04,SOAP,HAND WASH,SANTOOR,SOAP
71814,8901399058561,2017-01-08,SOAP,HAND WASH,SANTOOR,SOAP
115762,8901399058561,2017-01-13,SOAP,HAND WASH,SANTOOR,SOAP
122086,8901399058561,2017-01-14,SOAP,HAND WASH,SANTOOR,SOAP
136645,8901399058561,2017-01-16,SOAP,HAND WASH,SANTOOR,SOAP
358903,8901399058561,2017-03-12,SOAP,HAND WASH,SANTOOR,SOAP
359028,8901399058561,2017-03-12,SOAP,HAND WASH,SANTOOR,SOAP


In [11]:
# Adding week number column
joined_df['WEEK_NUM'] = joined_df['CREATED_STAMP'].dt.week
joined_df['DAY_NUM'] = joined_df['CREATED_STAMP'].dt.dayofyear

joined_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,BARCODE,CREATED_STAMP,CATEGORY_DESC,SUBCATEGORY_DESC,BRAND_DESC,BASEPACK,WEEK_NUM,DAY_NUM
1,8901399058561,2017-01-01,SOAP,HAND WASH,SANTOOR,SOAP,52,1
13695,8901399058561,2017-01-02,SOAP,HAND WASH,SANTOOR,SOAP,1,2
24018,8901399058561,2017-01-25,SOAP,HAND WASH,SANTOOR,SOAP,4,25
43420,8901399058561,2017-01-04,SOAP,HAND WASH,SANTOOR,SOAP,1,4
71814,8901399058561,2017-01-08,SOAP,HAND WASH,SANTOOR,SOAP,1,8
115762,8901399058561,2017-01-13,SOAP,HAND WASH,SANTOOR,SOAP,2,13
122086,8901399058561,2017-01-14,SOAP,HAND WASH,SANTOOR,SOAP,2,14
136645,8901399058561,2017-01-16,SOAP,HAND WASH,SANTOOR,SOAP,3,16
358903,8901399058561,2017-03-12,SOAP,HAND WASH,SANTOOR,SOAP,10,71
359028,8901399058561,2017-03-12,SOAP,HAND WASH,SANTOOR,SOAP,10,71


In [12]:
# Removing the category "Others" so that association rule mining using Apriori

In [13]:
no_of_weeks = 53;
joined_df_copy = joined_df.copy(deep=True)

with open('weekly_data.csv', 'w') as weekly_file:
    weekly_csv = csv.writer(weekly_file)
    for x in range(1,no_of_weeks):
        weekly_df = joined_df_copy.where(joined_df_copy.WEEK_NUM == x)
        weekly_data = weekly_df['CATEGORY_DESC'].tolist()
        weekly_data = [x for x in weekly_data if not((str(x)=='nan' or str(x)=='OTHERS'))]
        weekly_csv.writerow(list(set(weekly_data)))
    

In [12]:
no_of_days = 31+28+31;
joined_df_copy = joined_df.copy(deep=True)

with open('daily_data.csv', 'w') as daily_file:
    daily_csv = csv.writer(daily_file)
    for x in range(1,no_of_days):
        daily_df = joined_df_copy.where(joined_df_copy.DAY_NUM == x)
        daily_data = daily_df['CATEGORY_DESC'].tolist()
        daily_data = [x for x in daily_data if not((str(x)=='nan' or str(x)=='OTHERS'))]
        daily_csv.writerow(list(set(daily_data)))

In [39]:
datas = ["", 'Divyaksh','Shukla','']
list(filter(None, datas))

['Divyaksh', 'Shukla']

# TensorFlow training

- open source tool to provide regression models
- Here, we used python to analyse the data

#### xlrd
- to extract data from xls

In [ ]:
import tensorflow as tf
import xlrd

rng = np.random

data_file = "new3.xls"
epochs = 100
display_step = 50

dfile = xlrd.open_workbook(data_file, encoding_override="utf-8")
sheet = dfile.sheet_by_index(0)
data = np.asarray([sheet.row_values(i)for i in range(1, sheet.nrows)])
new_data = np.hsplit(data , 2)
# print(new_data[0])

**barcodes** are in new_data[0]
**created_stamps** are in new_data[1]

In [ ]:
barcode = new_data[0]
created_stamps = new_data[1]

In [ ]:
####creating placeholders

BARCODE =tf.placeholder(tf.float32,name="BARCODE")
CREATED_STAMP =tf.placeholder(tf.float32,name="CREATED_STAMP")

####creating weight and bias, initialized to 0

w =tf.Variable(rng.randn(),name="BARCODE")
b =tf.Variable(rng.randn(),name="CREATED_STAMP")

Y_pred = BARCODE * w + b

#### Mean squared error
cost = tf.reduce_sum(tf.pow(Y_pred-CREATED_STAMP, 2))/(2*n_samples)

#loss = tf.square(CREATED_STAMP-Y_pred, name="loss")

opt = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(epochs):
        for x, y in data:
            sess.run(opt, feed_dict={BARCODE: x, CREATED_STAMP: y})

        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            c = sess.run(cost, feed_dict={BARCODE: x, CREATED_STAMP: y})
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "w=", sess.run(w), "b=", sess.run(b))

    w_value, b_value = sess.run([w, b])
    print("Optimization finished!!")
    training_cost = sess.run(cost, feed_dict={BARCODE: x, CREATED_STAMP: y})
    print("Training cost=", training_cost, "w=", sess.run(w), "b=", sess.run(b), '\n')

    # Graphic display
    plt.plot(x, y, 'ro', label='barcode vs timestamp')
    plt.plot(x, sess.run(w) * x + sess.run(b), label='Fitted line')
    plt.legend()
    plt.show()

### Graphic Data

The data is plotted on a matplotlib library. 